In [1]:
from transformers import DistilBertForSequenceClassification,Trainer, TrainingArguments,DistilBertTokenizerFast
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import torch
import spacy
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import os
import asyncio
from openai import AsyncOpenAI


c:\Users\nadim\miniconda3\envs\tensorflowgpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def send_message(receiver, content, **kwargs):
    return f"Sent to \"{receiver}\" the message: \"{content}\""

def ask_RAG(question):
    return f"Asked to RAG: \"{question}\"\nThe RAG replied: \"I don't know\""

In [4]:
# Assume send_message and ask_RAG are defined elsewhere
model = spacy.load("./output/model-best")

def format_api_call(message, model):
    # Process the text with the trained model
    doc = model(message)
    
    # Extract entities for "person" and "content"
    receiver = []
    content = []
    for ent in doc.ents:
        if ent.label_ == "person":
            receiver.append(ent.text)
        elif ent.label_ == "content":
            content.append(ent.text)
    
    # Format the API call JSON
    api_call = {
        "job": "send_message",
        "receiver": " ".join(receiver),
        "content": " ".join(content)
    }
    return api_call

def process_row(row):
    """
    Processes a single row of the DataFrame.
    Depending on the 'label_text', it either sends a message or asks RAG.
    """
    question = row["question"]
    label_text = row["label_text"]
    
    if label_text == "send_message":
        formatted_call = format_api_call(question, model)  # Assuming 'model' is defined globally or passed another way
        return send_message(**formatted_call)
    elif label_text == "question_rag":
        return ask_RAG(question)
    else:
        return None

def main():
    df = pd.read_csv('question_classif.csv')
    results = df.apply(process_row, axis=1)

    # Assuming you want to print the results for each row
    for result in results:
        print(result)

if __name__ == "__main__":
    main()


Asked to RAG: "What are the recommended prerequisites for the introduction to machine learning course?"
The RAG replied: "I don't know"
Asked to RAG: "Does the cybersecurity course cover intrusion detection methods?"
The RAG replied: "I don't know"
Asked to RAG: "How can I enroll in the Python course?"
The RAG replied: "I don't know"
Asked to RAG: "What are the main basic concepts covered in the SQL course?"
The RAG replied: "I don't know"
Asked to RAG: "Does the React course include practical projects to apply the learned concepts?"
The RAG replied: "I don't know"
Asked to RAG: "What are the main topics covered in the introduction to machine learning course?"
The RAG replied: "I don't know"
Asked to RAG: "Does the cybersecurity course address the latest trends in cybersecurity?"
The RAG replied: "I don't know"
Asked to RAG: "Are there any recommended additional resources to deepen my knowledge in Python?"
The RAG replied: "I don't know"
Asked to RAG: "Does the React course cover the u

Process data

train model last two layers

In [5]:
from transformers import TFDistilBertModel, DistilBertConfig
import tensorflow as tf

# Charger la configuration et le modèle
config = DistilBertConfig.from_pretrained('distilbert-base-uncased', num_labels=2)
model = TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config)

# Construire un modèle personnalisé qui inclut DistilBERT et ajoute une couche Dense pour la classification
input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="attention_mask")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [6]:
# D'abord, geler toutes les couches
model.distilbert.trainable = False

# Obtenez d'abord la référence à la couche transformer de DistilBERT
transformer_layer = model.distilbert.transformer

# Ensuite, accédez à la 5ème couche transformer
fifth_layer = transformer_layer.layer[4]

# Rendez entraînables uniquement les sous-couches spécifiques de la 5ème couche
fifth_layer.ffn.trainable = True  # Le réseau feed-forward
fifth_layer.output_layer_norm.trainable = True  # La couche de normalisation de sortie

# Vérifiez les paramètres entraînables
print(f"Layer 5 FFN trainable: {fifth_layer.ffn.trainable}")
print(f"Layer 5 Output Layer Norm trainable: {fifth_layer.output_layer_norm.trainable}")

Layer 5 FFN trainable: True
Layer 5 Output Layer Norm trainable: True


In [7]:
# Obtenir les sorties de DistilBERT
distilbert_output = model(input_ids=input_ids, attention_mask=attention_mask)
hidden_state = distilbert_output.last_hidden_state  # Les représentations cachées de la dernière couche

# Sélectionner la représentation du premier token (CLS token) pour la classification
cls_token = hidden_state[:, 0, :]

# Ajouter la couche Dense pour la classification
output = tf.keras.layers.Dense(1, activation='sigmoid')(cls_token)

# Créer le nouveau modèle en définissant correctement les inputs et outputs
new_model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

# Compiler le modèle
new_model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Afficher le résumé du modèle pour vérification
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_ids[0][0]',              
 BertModel)                     ast_hidden_state=(N               'attention_mask[0][0]']         
                                one, None, 768),                                                  
                                 hidden_states=None                                           

In [20]:
from transformers import DistilBertTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Load your dataset
dataset_path = 'question_classif.csv'
dataset = pd.read_csv(dataset_path)

# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the dataset
tokenized_data = tokenizer(list(dataset['question']), padding=True, truncation=True, return_tensors="tf")

# Convert TensorFlow tensors to NumPy arrays for compatibility with train_test_split
input_ids = tokenized_data['input_ids'].numpy()
attention_mask = tokenized_data['attention_mask'].numpy()
labels = dataset['label'].to_numpy()

# Splitting the dataset into training and validation sets (correctly using the converted arrays)
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(
    input_ids, labels, random_state=42, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(
    attention_mask, labels, random_state=42, test_size=0.1)

# Convert splits to TensorFlow datasets for training
train_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': train_inputs, 'attention_mask': train_masks},
    train_labels
)).batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': validation_inputs, 'attention_mask': validation_masks},
    validation_labels
)).batch(32)

# Train the model
history = new_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)


Epoch 1/10
3/3 [==============================] - 6s 691ms/step - loss: 0.7077 - accuracy: 0.5000 - val_loss: 0.6891 - val_accuracy: 0.5000
Epoch 2/10
3/3 [==============================] - 0s 44ms/step - loss: 0.6594 - accuracy: 0.5455 - val_loss: 0.6213 - val_accuracy: 0.5000
Epoch 3/10
3/3 [==============================] - 0s 46ms/step - loss: 0.6077 - accuracy: 0.7841 - val_loss: 0.5662 - val_accuracy: 0.9000
Epoch 4/10
3/3 [==============================] - 0s 44ms/step - loss: 0.5500 - accuracy: 0.9432 - val_loss: 0.5196 - val_accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 0s 43ms/step - loss: 0.5062 - accuracy: 0.9773 - val_loss: 0.4776 - val_accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 0s 47ms/step - loss: 0.4753 - accuracy: 1.0000 - val_loss: 0.4381 - val_accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 0s 49ms/step - loss: 0.4469 - accuracy: 1.0000 - val_loss: 0.4017 - val_accuracy: 1.0000
Epoch 8/10
3/3 [==========

Part 2 TD5

In [33]:
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import spacy  # Assuming you're using spaCy for the NER model
import asyncio
import nest_asyncio
nest_asyncio.apply()

# Load the tokenizer and the DistilBERT model for sequence classification
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
sequence_classification_model = new_model

# Load your trained NER model (assuming it's a spaCy model)
ner_model = spacy.load("./output/model-best")


def predict_intent(text):
    inputs = tokenizer.encode_plus(text, return_tensors="tf", max_length=512, truncation=True, padding="max_length")
    outputs = new_model.predict({'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask']})
    
    # Since the model output is a probability (due to sigmoid activation), use a threshold to determine the class
    prediction = (outputs > 0.5).astype(int)  # Convert boolean to int (True to 1, False to 0)
    prediction = prediction[0][0]  # Adjust indexing based on your model's output shape
    
    return "Question" if prediction == 1 else "Other"


def process_question_with_ner(text):
    # Use the NER model to process the text
    #doc = ner_model(text)
    # Extract relevant entities or information as needed
    # Placeholder logic for processing; adapt as necessary
    #entities = [(ent.text, ent.label_) for ent in doc.ents]
    entities = format_api_call(text, ner_model)
    return f"{send_message(**entities)}"

async def ask_rag(question):
    print('-----QUESTION 1-----',question)
    async with AsyncOpenAI(api_key="Removed for security, add yours please") as client:
        chat_completion = await client.chat.completions.create(
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": question},
            ],
            model="gpt-3.5-turbo",
        )
        # Access the 'content' attribute directly using dot notation
        return chat_completion.choices[0].message.content

def send_virtual_assistant(user_input):
    intent = predict_intent(user_input)
    if intent == "Question":
        ner_result = process_question_with_ner(user_input)
        return f'Processed with NER model: "{ner_result}"'
    else:
        # Use asyncio's get_event_loop() and run_until_complete for environments with an existing loop
        loop = asyncio.get_event_loop()
        if loop.is_running():
            future = asyncio.ensure_future(ask_rag(user_input))
            rag_answer = loop.run_until_complete(future)
        else:
            rag_answer = asyncio.run(ask_rag(user_input))
        return f'RAG answer: "{rag_answer}"'

# Example test cases
test_inputs = [
    "Ask the python teacher when is the next class",
    "What are the pre-requisites for the python class?",
    "Schedule a meeting with the data science teacher next week"
]

# Example usage
for input_text in test_inputs:
    response = send_virtual_assistant(input_text)
    print(f"Input: {input_text}\nResponse: {response}\n")

1/1 [==============================] - 0s 45ms/step
Input: Ask the python teacher when is the next class
Response: Processed with NER model: "Sent to "the python teacher" the message: "when next class""

1/1 [==============================] - 0s 43ms/step
-----QUESTION 1----- What are the pre-requisites for the python class?
Input: What are the pre-requisites for the python class?
Response: RAG answer: "The prerequisites for a Python class may vary depending on the specific course and level of proficiency expected. However, some common prerequisites for a beginner or introductory Python class may include:

1. Basic understanding of computer operation and file management.
2. Familiarity with fundamental programming concepts such as variables, data types, loops, and conditional statements.
3. Knowledge of using a text editor or integrated development environment (IDE).
4. Basic understanding of mathematical concepts.
5. Comfort with problem-solving and logical thinking.

It's recommended